## Feature Pipeline for the `exploded_swells` feature group

This feature pipeline can be run on a schedule using github actions (see github repo for the example file).

### Requirements

 * hopsworks

In [1]:
#!pip install -U hopsworks

In [1]:
import os
import urllib.request  
import re
from itertools import chain
import pandas as pd
import numpy as np
import hopsworks
from datetime import timedelta
from datetime import datetime

### Not app.hopsworks.ai ?

If you are running your own Hopsworks cluster (not app.hopsworks.ai):

 * uncomment the cell below
 * fill in details for your cluster
 * run the cel

In [2]:
#key=""
#with open("api-key.txt", "r") as f:
#    key = f.read().rstrip()
#os.environ['HOPSWORKS_PROJECT']="dowlingj"
#os.environ['HOPSWORKS_HOST']="35.187.178.84"
#os.environ['HOPSWORKS_API_KEY']=key    

### Backfill the feature group 

If you set `BACKFILL` to `True` in the cell below, and continue running all the cells, you will insert swell predictions from the `swells-clean.csv` file into the feature group.

When `BACKFILL` is `False`, it will download the latest predictions from the NOA 62081 Buoy and insert them into the feature group.

In [3]:
# Training columns - height, period, direction, hits_at
# Prediction columns - pred_dtime', 'hour', 'pred_day', 'pred_hour
# event_time column - hits_at
# primary key - beach_id

BACKFILL=True
if os.environ.get('CJSURF_BACKFILL') == "False":
    BACKFILL=False
hours=119
version=1
buoy = "62081"
backfill_url="https://repo.hops.works/master/hopsworks-tutorials/data/cjsurf/swells-clean.csv"        

### Understand the Features

We store 119*4=476 columns in the `swell_predictions` feature group. It is 119 different swell predictions, one for each hour from hour=0, hour=2, ..., hour=238.  Each prediction is made using the `height`, `period`, and `direction` features. The `hits_at` feature is used to estimate the time at which the swell arrives at Lahinch beach.

Parse the data in the URL managed by NOA containing the predictions for the Buoy:

https://polar.ncep.noaa.gov/waves/WEB/gfswave.latest_run/plots/gfswave.62081.bull

In [4]:
# Used to scrap latest swell predictions
# Format for date is 'YYYYMMDD', e.g., 20220817
def get_latest_url(today):
    pred_date = today.strftime("%Y%m%d")
    pred_date = "20220820"

    # There are 4 predictions per day at hours: "00", "06", "12", "18",
    h=int(today.strftime("%H"))
    found = False
    test_url = ""
    while not found:
        pred_hour = "00"
        if h > 5:
            pred_hour = "06"
        if h > 11:
            pred_hour = "12" 
        if h > 17:
            pred_hour = "18"
        test_url = "https://ftpprd.ncep.noaa.gov/data/nccf/com/gfs/prod/gfs." + pred_date + \
      "/" + pred_hour + "/wave/station/bulls.t" + pred_hour + "z/gfswave." + buoy + ".bull"
        print("test_url")
        try:
            urllib.request.urlopen(test_url)
            found = True
        except urllib.error.HTTPError as e: 
            # assume 404, URL not found. Try previous time.
            h = h - 6
            if h < 0:
                print("ERROR: Could not download url: " + test_url)
                exit()
    return test_url, pred_hour


In [5]:
def process_url(buoy_url):
    out = []
    c=0
    for line in urllib.request.urlopen(buoy_url):
        l = line.decode('utf-8') #utf-8 or iso8859-1 or whatever the page encoding scheme is
        row=[]
        if "Cycle" in l:
            # Parse this line "Cycle    : 20220818  6 UTC"
            regex = re.findall(r'Cycle.*:\s+([0-9]+)\s+([0-9]+)\s+UTC.*', l)
            if len(regex):
                thedate=regex[0]
        else:
            res = re.match(r'.*[|]\s+([0-9]+)\s+([0-9]+)\s+[|].*', l)
            waves = re.findall(r'[|][\s\*]+([0-9\.]+)\s+([0-9\.]+)\s+([0-9]+)\s+[|]', l)
            if res is not None:
                row.append(thedate)
                row.append(res.groups())
            if len(waves):
                if len(waves) > 3:
                    # print("found > 3 waves, reduce to 3")
                    waves = waves[:3]
                b = []
                list(b.extend(item) for item in waves)
                row.append(b)
                my = tuple(chain.from_iterable(row))
                out.append(my)
    return out, thedate

### Feature engineering - select the best swell for Lahinch

Each hour with a prediction at https://polar.ncep.noaa.gov/waves/WEB/gfswave.latest_run/plots/gfswave.62081.bull can contain zero to many different swells. Extract the swell that is gives the expected highest surf at Lahinch, based on the angle of the swell direction (Lahinch has a swell direction window of around 20 degrees to 120 degrees.

In [6]:
scraped_columns=['pred_dtime', 'hour', 'pred_day', 'pred_hour', 'height1', 'period1', 'direction1', 'height2', 
         'period2', 'direction2', 'height3', 'period3', 'direction3'] 

def is_valid_swell_direction(direction):
    if int(direction) > 180 or int(direction) < 20:
        return False
    return True

def best_height(row):
    best_secondary=2
    # Check which is best secondary swell - swell 2 or swell 3?
    if row['direction3'] != None:
        if is_valid_swell_direction(row['direction3']):
            if is_valid_swell_direction(row['direction2']) == False :
                best_secondary=3    
    best_direction = "direction" + str(best_secondary)
    best=1
    # Check which is best of swell 1 and secondary swell ?
    if row[best_direction] != None and is_valid_swell_direction(row[best_direction]) == True:
        if is_valid_swell_direction(row['direction1']) == False:
            best=best_secondary
                
    height = row['height' + str(best)]
    period = row['period' + str(best)]
    direction = row['direction' + str(best)]
        
    return pd.Series([height, period, direction])

# feature engineering - estimate the time at which the swell arrives at Lahinch from buoy
def estimate_hits_at(row):
    # baseline estimate
    hits_at = row['pred_dtime'] + row['hour_offset'] + timedelta(hours=8) 
    
    if float(row['direction']) < 80 and float(row['direction']) > 66:
        hits_at = hits_at - timedelta(hours=1)
    if float(row['direction']) <= 66 and float(row['direction']) > 50:
        hits_at = hits_at - timedelta(hours=2)
    if float(row['direction']) <= 50 and float(row['direction']) > 20:
        hits_at = hits_at - timedelta(hours=3)
    if float(row['period']) > 12:
        hits_at = hits_at - timedelta(hours=1)
    
    return pd.Series([hits_at])
    

if BACKFILL == True:
    df = pd.read_csv(backfill_url, parse_dates=['hits_at', 'pred_dtime'])
#     num_rows = df.shape[0]
#     print("num_rows: " + str(num_rows))
#     rows = []
#     for i in range(1, num_rows):
#         row=[]
#         for j in range(0, len(secondary_columns)):
#             row.append("")
#         if i % 2 == 0:
#             rows.append(row)
#    df_secondary = pd.DataFrame(rows, columns=secondary_columns)
#    df = pd.concat([df, df_secondary],axis=1, join="outer")    
    
else: # BACKFILL == False
    today = datetime.now()
    url, pred_hour = get_latest_url(today)
    print(int(pred_hour))
    res,thedate=process_url(url)
    df = pd.DataFrame(res, columns=scraped_columns)
    df['pred_dtime'] = pd.to_datetime(df['pred_dtime'], format='%Y%m%d')
#    df['pred_dtime']  = df['pred_dtime']  + timedelta(hours=int(pred_hour)) 
    df.insert(loc=0, column="hour_offset", value=(df.reset_index().index*2))
    df['hour_offset'] = df.hour_offset.astype('timedelta64[h]')
    df['pred_dtime'] = df['pred_dtime'] + df.hour.astype('timedelta64[h]')
    df[['height','period','direction']]=df.apply(best_height, axis=1)
    df[['hits_at']]=df.apply(estimate_hits_at, axis=1)
    df['beach_id'] = 1
    df.drop(['height1', 'period1', 'direction1', 'height2', 'period2', 'direction2', 'hour_offset',
              'height3', 'period3', 'direction3','hour', 'pred_day', 'pred_hour'], axis=1, inplace=True) 

df

,beach_id,height,period,direction,hits_at,pred_dtime
0,1,3.1,11.2,87,2004-03-18 08:00:00,2004-01-01
1,1,3.4,11.2,83,2004-03-18 07:00:00,2004-01-01
2,1,2.7,10.4,80,2004-05-17 06:00:00,2004-01-01
3,1,3.4,11.2,75,2004-03-18 05:00:00,2004-01-01
4,1,3.4,11.3,78,2004-03-18 03:00:00,2004-01-01
...,...,...,...,...,...,...
13143,1,1.6,11.3,118,2005-10-18 14:00:00,2004-01-01
13144,1,1.4,11.4,118,2005-10-18 16:00:00,2004-01-01
13145,1,1.3,11.4,119,2005-10-18 17:00:00,2004-01-01
13146,1,1.3,11.4,119,2005-10-18 18:00:00,2004-01-01


In [8]:
df['pred_hour']=0

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13148 entries, 0 to 13147
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   beach_id    13148 non-null  int64         
 1   height      13148 non-null  float64       
 2   period      13148 non-null  float64       
 3   direction   13148 non-null  int64         
 4   hits_at     13148 non-null  datetime64[ns]
 5   pred_dtime  13148 non-null  datetime64[ns]
 6   pred_hour   13148 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(3)
memory usage: 719.2 KB


In [10]:
df.to_csv("swells-clean.csv", index=False)

In [12]:
res = df.duplicated(subset=['hits_at'])
res

0        False
1        False
2        False
3        False
4        False
         ...  
13143    False
13144    False
13145    False
13146    False
13147    False
Length: 13148, dtype: bool

In [10]:
df

,pred_dtime,height,period,direction,hits_at,beach_id
0,2022-08-20 18:00:00,2.31,9.9,118,2022-08-21 02:00:00,1
1,2022-08-20 18:00:00,2.23,9.8,118,2022-08-21 04:00:00,1
2,2022-08-20 18:00:00,2.16,9.7,119,2022-08-21 06:00:00,1
3,2022-08-20 18:00:00,2.10,9.7,119,2022-08-21 08:00:00,1
4,2022-08-20 18:00:00,2.05,9.6,120,2022-08-21 10:00:00,1
...,...,...,...,...,...,...
380,2022-08-20 18:00:00,1.63,11.6,138,2022-09-21 18:00:00,1
381,2022-08-20 18:00:00,1.69,11.5,138,2022-09-21 20:00:00,1
382,2022-08-20 18:00:00,1.78,11.3,141,2022-09-21 22:00:00,1
383,2022-08-20 18:00:00,1.75,11.2,137,2022-09-22 00:00:00,1


In [11]:
# matches = ["height", "period", "direction", "hits_at"]

# if BACKFILL == False:
#     entry = []
#     data = []
#     for index, row in df.iterrows():
#         if (index==0):
#             data.append(row['beach_id'])
#             data.append(row['pred_dtime'])
#         if (index < hours):
#             for m in matches:
#                 data.append(row[m])

#     entry.append(data)
#     first_columns=['beach_id', 'pred_dtime', 'height', 'period', 'direction', 'hits_at']    
#     all_columns = first_columns + secondary_columns
#     df2 = pd.DataFrame(entry, columns=all_columns)
# else:    
#     df2=df

# for i in range(1,hours):
#     for j in matches:
#       df2[j+str(i*2)] = pd.to_numeric(df2[j+str(i*2)]).astype(np.float64)
# df2

### Connect to your Hopsworks cluster

If you only set the HOPSWORKS_API_KEY, it will assume you are connecting to app.hopsworks.ai.
Set HOPSWORKS_HOST and HOPSWORKS_PROJECT environment variables to connect to a different Hopsworks cluster.

In [12]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/398
Connected. Call `.close()` to terminate connection gracefully.


Write your features to the `swells_exploded` feature group.

In [13]:
swells_fg = fs.get_or_create_feature_group(name="swells_exploded",
                version=version,
                primary_key=["beach_id"],
                event_time="hits_at",
                description="Buoy surf height predictions",
                statistics_config={"enabled": True, "histograms": True, "correlations": True}
                )
swells_fg.insert(df)
    

Uploading Dataframe: 0.00% |          | Rows 0/385 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/swells_exploded_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fa05818da30>, None)